# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
!jupyter nbextension enable --py gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import vacation_key


ModuleNotFoundError: No module named 'config'

In [4]:
data = pd.read_csv('../VacationPy/cities_weather_data.csv')

data.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Butaritari,32,KI,1579611058,64,3.07,172.79,84.79,2.46
1,Smithers,90,CA,1579611059,92,54.78,-127.17,19.40,4.70
2,Severo-Kuril'sk,100,RU,1579611060,90,50.68,156.12,13.84,7.85
3,Chokurdakh,100,RU,1579611061,97,70.63,147.92,-24.09,8.79
4,Upernavik,100,GL,1579611062,57,72.79,-56.15,4.51,6.62


In [5]:
places = data[["Lat", "Lng"]]

rating = data["Humidity"].astype(float)

In [6]:
# Plot Heatmap
gmaps.configure(api_key=vacation_key)
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(places, weights=rating, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
city_df = data.copy()
city_df = city_df[city_df['Max Temp'] >= 70]
city_df = city_df[city_df['Max Temp'] <= 80]  
city_df = city_df[city_df['Wind Speed'] <= 10]
city_df = city_df[city_df['Cloudiness'] == 0]
city_df = city_df.dropna()
city_df = city_df.reset_index()
city_df

,index,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,105,Ormara,0,PK,1579611178,48,25.21,64.64,70.11,3.38
1,122,Moree,0,AU,1579611198,25,-29.47,149.85,77.00,3.36
2,141,Dwarka,0,IN,1579611219,50,22.24,68.97,74.25,8.21
3,189,Viedma,0,AR,1579611274,41,-40.81,-63.00,73.29,4.63
4,229,Trelew,0,AR,1579611319,24,-43.25,-65.31,79.12,6.11
5,232,Mecca,0,SA,1579611101,35,21.43,39.83,76.12,8.55
6,270,Narrabri,0,AU,1579611367,31,-30.32,149.78,78.01,9.17
7,275,General Roca,0,AR,1579611373,50,-39.03,-67.58,75.20,6.93
8,379,Adrar,0,MR,1579611491,27,20.50,-10.07,70.97,6.71
9,503,Mae Sai,0,TH,1579611635,57,20.43,99.88,75.20,2.24


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = city_df

hotel_df["Hotel Name"] = np.nan

hotel_df = hotel_df.drop(columns='index')

hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Ormara,0,PK,1579611178,48,25.21,64.64,70.11,3.38,NaN
1,Moree,0,AU,1579611198,25,-29.47,149.85,77.00,3.36,NaN
2,Dwarka,0,IN,1579611219,50,22.24,68.97,74.25,8.21,NaN
3,Viedma,0,AR,1579611274,41,-40.81,-63.00,73.29,4.63,NaN
4,Trelew,0,AR,1579611319,24,-43.25,-65.31,79.12,6.11,NaN
5,Mecca,0,SA,1579611101,35,21.43,39.83,76.12,8.55,NaN
6,Narrabri,0,AU,1579611367,31,-30.32,149.78,78.01,9.17,NaN
7,General Roca,0,AR,1579611373,50,-39.03,-67.58,75.20,6.93,NaN
8,Adrar,0,MR,1579611491,27,20.50,-10.07,70.97,6.71,NaN
9,Mae Sai,0,TH,1579611635,57,20.43,99.88,75.20,2.24,NaN


In [9]:
hotels = []
#checker variable
i = 0

# for loop to search through city coords
for cities in hotel_df['City']:
    
    target_coordinates = str(hotel_df['Lat'][i])+' , '+str(hotel_df['Lng'][i])
    
    i+=1
    
    # target_radius = 5000
    
    # target_type = 'lodging'
    
    params = {
    "location": target_coordinates,
    "radius": 5000,
    "type": 'lodging',
    "key": vacation_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# try/except to check if a city turns up    
    
    try:
        response = requests.get(base_url, params=params)
        hotel_results = response.json()
        hotels.append(hotel_results['results'][0]['name'])
        
    except IndexError:
        hotels.append(np.nan)

#append dataframe with hotel names
hotel_df['Hotel Name'] = hotels

hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Ormara,0,PK,1579611178,48,25.21,64.64,70.11,3.38,NaN
1,Moree,0,AU,1579611198,25,-29.47,149.85,77.00,3.36,Molika Springs Motel
2,Dwarka,0,IN,1579611219,50,22.24,68.97,74.25,8.21,Hotel Narayan Avezika Comfort
3,Viedma,0,AR,1579611274,41,-40.81,-63.00,73.29,4.63,Hotel casa crespo
4,Trelew,0,AR,1579611319,24,-43.25,-65.31,79.12,6.11,Hotel Touring Club
5,Mecca,0,SA,1579611101,35,21.43,39.83,76.12,8.55,Hotel Fairmont Makkah Clock Royal Tower
6,Narrabri,0,AU,1579611367,31,-30.32,149.78,78.01,9.17,Crossroads Hotel
7,General Roca,0,AR,1579611373,50,-39.03,-67.58,75.20,6.93,Hotel El Recreo
8,Adrar,0,MR,1579611491,27,20.50,-10.07,70.97,6.71,NaN
9,Mae Sai,0,TH,1579611635,57,20.43,99.88,75.20,2.24,Piyaporn Hill Paradise Hotel


In [10]:
hotel_df = hotel_df.dropna()

hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
1,Moree,0,AU,1579611198,25,-29.47,149.85,77.00,3.36,Molika Springs Motel
2,Dwarka,0,IN,1579611219,50,22.24,68.97,74.25,8.21,Hotel Narayan Avezika Comfort
3,Viedma,0,AR,1579611274,41,-40.81,-63.00,73.29,4.63,Hotel casa crespo
4,Trelew,0,AR,1579611319,24,-43.25,-65.31,79.12,6.11,Hotel Touring Club
5,Mecca,0,SA,1579611101,35,21.43,39.83,76.12,8.55,Hotel Fairmont Makkah Clock Royal Tower
6,Narrabri,0,AU,1579611367,31,-30.32,149.78,78.01,9.17,Crossroads Hotel
7,General Roca,0,AR,1579611373,50,-39.03,-67.58,75.20,6.93,Hotel El Recreo
9,Mae Sai,0,TH,1579611635,57,20.43,99.88,75.20,2.24,Piyaporn Hill Paradise Hotel
10,Ciudad Lazaro Cardenas,0,MX,1579611637,78,17.96,-102.20,73.18,4.29,Hotel Quinta Antigua
11,Parkes,0,AU,1579611688,58,-33.13,148.18,72.00,4.50,Station Motel


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# Add marker layer ontop of heat map
fig = gmaps.figure()

markers = gmaps.marker_layer(places, info_box_content=hotel_info)

fig.add_layer(markers)

fig

# Display Map

Figure(layout=FigureLayout(height='420px'))